In [1]:
import numpy as np
from pandas import read_csv

# from average_atom.core.average_atom_new import AverageAtom as AA
from average_atom.core.aa_types import AverageAtomFactory
from average_atom.core.misc import jacobi_relaxation, sor
from average_atom.core.grids import NonUniformGrid
from average_atom.core.physics import ThomasFermi, FermiDirac

import matplotlib.pyplot as plt

from hnc.hnc.constants import *
from hnc.hnc.misc import rs_from_n, n_from_rs

plt.rcParams.update({'font.size': 10})
plt.rcParams["figure.figsize"] = (5,3.5)
plt.rcParams["figure.dpi"] = 200
plt.rcParams['xtick.labelsize']=14

color_cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']

ModuleNotFoundError: No module named 'mendeleev'


In [2]:
# Aluminum
name='Al'
Z, A = 13, 27 

ρ_solid = 2.699 #g/cc, the solid density of aluminum 
ρ = ρ_solid#8.1 # g/cc
ni_cc = ρ/(A*amu_to_AU*AU_to_g)
ni_AU = ni_cc*invcc_to_AU
rs = rs_from_n(ni_AU)

R  = 10*rs

Ti_eV = 1 
Te_eV = 1


#

Te_AU = Te_eV*eV_to_AU
Ti_AU = Ti_eV*eV_to_AU


aa_ZJ  = AverageAtomFactory.create_model("ZJ_ISModel", Z, A, Ti_AU, Te_AU, rs, R, name=name, ignore_vxc=True, xc_type='KSDT', Npoints=2000, rmin=1e-4, N_stencil_oneside = 2)
# aa_ZJ_cs  = AverageAtomFactory.create_model("ZJ_CSModel", Z, A, Ti_AU, Te_AU, rs, R, name=name, ignore_vxc=False, Npoints=2000, rmin=1e-4, N_stencil_oneside = 2)
# aa_CS  = AverageAtomFactory.create_model("TFStarret2014", Z, A, Ti_AU, Te_AU, rs, R, name=name, ignore_vxc=False, Npoints=2000, rmin=1e-4, N_stencil_oneside = 2)

________________________
Generating NPA
T:  0.03674932539796232
Using More TF fit for initial Zstar = 2.495
	Intializing grid
Initializing, True
Initializing
	Creating Integral Equation Solver
Intialized Potentials and Densities
Initialized


/home/zach/plasma/average_atom/core/average_atom_new.py:259: RuntimeWarning: invalid value encountered in power
  nb_Mavrin_Demura_func = lambda r: np.nan_to_num(self.Z/(4*π*r_TF**3) * (Φ_Mavrin_Demura(x(r))/x(r) )**1.5)


In [3]:
%%time
aa_ZJ.solve(verbose=True)
aa_ZJ.set_uii_eff()

Beginning self-consistent electron solver.
_________________________________
__________________________________________
TF Iteration 0
	μ = -2.248224108e-01, change: 1.598471249e+00 (converged=False)
	φe Err =  5.605e-11, φe change =  7.588e-01
	ne Err =  6.289e-02, ne change =  1.770e-05
	Q =  0.000e+00 -> -3.487e+00, 
	Zstar guess =  2.495e+00. Current Zstar:  2.495e+00 (converged=True)
	Change =  2.104e+00
__________________________________________
TF Iteration 1
	μ = -2.242056176e-01, change: 2.743468579e-03 (converged=False)
	φe Err =  5.913e-11, φe change =  5.566e-02
	ne Err =  9.074e-02, ne change =  1.815e-05
	Q = -3.487e+00 ->  1.073e+00, 
	Zstar guess =  2.495e+00. Current Zstar:  2.495e+00 (converged=True)
	Change =  6.125e-02
__________________________________________
TF Iteration 2
	μ = -2.237022539e-01, change: 2.245098695e-03 (converged=False)
	φe Err =  5.837e-11, φe change =  1.706e-02
	ne Err =  3.237e-02, ne change =  1.753e-06
	Q =  1.073e+00 ->  2.889e+00, 
	Zstar


KeyboardInterrupt



In [4]:
%%time
aa_ZJ_cs.solve(picard_alpha=0.5,verbose=True)
aa_ZJ_cs.set_uii_eff()

NameError: name 'aa_ZJ_cs' is not defined

In [5]:
%%time
aa_CS.solve(verbose=True)
aa_CS.make_iet()
aa_CS.set_uii_eff()

NameError: name 'aa_CS' is not defined

In [6]:
aa_CS.iet.dst_type=3


KeyboardInterrupt



In [ ]:
from importlib import reload
try:
    reload(plotting)
except NameError as e:
    from average_atom.core import plotting as plotting
    print("Error: ",e)
except Exception as e:
    print("Unexpected error during reload")
    print(e)
    
from average_atom.core.plotting import compare_aa, plot_convergence, plot_Uei, plot_Uii, plot_hii

fig, axs = plot_Uei([aa_CS, aa_ZJ, aa_ZJ_cs])
fig, axs = plot_Uii([aa_CS, aa_ZJ, aa_ZJ_cs])


In [ ]:
from scipy.interpolate import PchipInterpolator
from pandas import read_csv

fig, axs = plot_hii([aa_CS, aa_ZJ, aa_ZJ_cs])

# Data from community TCCW
upper_data = read_csv("/home/zach/plasma/hnc/TCCW/Final_TCCW/Fig5c_Upper.txt", delim_whitespace=True, header=1)
lower_data = read_csv("/home/zach/plasma/hnc/TCCW/Final_TCCW/Fig5c_Lower.txt", delim_whitespace=True, header=1)

ax = axs[0]
ax.fill_between(lower_data['r/ri'],lower_data['gii'], PchipInterpolator(upper_data['r/ri'], upper_data['gii'])(lower_data['r/ri']), alpha=0.5, color='k', label='Workshop DFT Spread')

ax = axs[1]
ax.fill_between(lower_data['r/ri'],lower_data['gii']-1, PchipInterpolator(upper_data['r/ri'], upper_data['gii'])(lower_data['r/ri'])-1, alpha=0.5, color='k', label='Workshop DFT Spread')


### Compare Density Profiles

In [ ]:
def plot_ne(ne_data, ks=False, figax = None, **kwargs):#line = None, color=None, label=None):
    if figax is None:
        fig, ax = plt.subplots()
    else:
        fig, ax  = figax
    
    fourpi = True
    if fourpi==True:
        if ks==False:
            ax.plot(ne_data[0], 4*π*ne_data[0]**2*ne_data[1], **kwargs )#, linestyle=line, color=color, label=label)
        else:
            ax.plot(ne_data[0], ne_data[1], **kwargs )#, linestyle=line, color=color, label=label)
        ax.set_ylabel(r"$4 \pi  r^2 n_{bound}$")        
    else:
        if ks==False:
            ax.plot(ne_data[0], ne_data[1], **kwargs )#, linestyle=line, color=color, label=label)
        else:
            ax.plot(ne_data[0], (4*π*ne_data[0]**2)**-1*ne_data[1], **kwargs )#, linestyle=line, color=color, label=label)
        ax.set_ylabel(r"$n_{free} $ [au]")        

    if True:
        ax.set_xlim(0,rs)
        ax.set_ylim(0, 13)
    else:
        ax.set_xlim(1e-2,30)
        ax.set_yscale('log')#, linthresh=1e-3)
        ax.set_xscale('log')
        ax.set_ylim(1e-3, 1e1)

    
    ax.set_xlabel(r"$r $ [au]")
    
    ax.legend(fontsize=6, ncols=2)
    
# Fixed Ti
fig, ax = plt.subplots()
ax.set_title(r"Aluminum @ $2.7$ g/cc, $T_i = 1 $ eV")

# plot_ne(NS_ks_ne11, ks=True, figax = (fig, ax), color=color_cycle[0], linestyle='-', label=r"NS QM $T_e=1$ eV")
# plot_ne(NS_tf_ne11, ks=False, figax = (fig, ax), color=color_cycle[0], linestyle='--', label=r"NS TF $T_e=1$ eV")
ZJ_csaa = np.array([aa_CS.grid.xs, aa_CS.ne_full])
ZJ_tfaa = np.array([aa_ZJ.grid.xs, aa_ZJ.ne])
ZJ_tfaa_cs = np.array([aa_ZJ_cs.grid.xs, aa_ZJ_cs.ne])
plot_ne(ZJ_csaa, ks=False, figax = (fig, ax), color=color_cycle[1], linestyle='-', marker='', markersize=1, label=r"ZJ (SS2014) TF $T_e=1$ eV $\langle Z \rangle$={0:0.2f}".format(aa_CS.Zstar))
plot_ne(ZJ_tfaa, ks=False, figax = (fig, ax), color=color_cycle[2], linestyle='--', marker='', markersize=1, label=r"ZJ (IS) TF $T_e=1$ eV $\langle Z \rangle$={0:0.2f}".format(aa_ZJ.Zstar))
plot_ne(ZJ_tfaa_cs, ks=False, figax = (fig, ax), color=color_cycle[3], linestyle=':', marker='', markersize=1, label=r"ZJ (CS) TF $T_e=1$ eV $\langle Z \rangle$={0:0.2f}".format(aa_ZJ.Zstar))
# ZJ_csaa = np.array([aa_CS.core_atom.grid.xs, aa_CS.core_atom.ne])
# plot_ne(ZJ_csaa, ks=False, figax = (fig, ax), color=color_cycle[1], linestyle='--', marker='.', markersize=1, label=r"ZJ (CS2014) TF $T_e=1$ eV")

In [ ]:
from scipy.interpolate import interp1d

fig, ax  = plt.subplots()

ZJ_csaa = np.array([aa_CS.grid.xs, aa_CS.ne_full])

# ax.plot(aa_CS.core_atom.grid.xs, aa_CS.grid.A_dfdx.dot(aa_CS.core_atom.φe  ), '--.')
ax.plot(aa_CS.core_atom.grid.xs, aa_CS.core_atom.grid.A_dfdx.dot(aa_CS.core_atom.φe + aa_CS.core_atom.φion ), '--.')
ax.plot(aa_CS.core_atom.grid.xs, (aa_CS.core_atom.φe + aa_CS.core_atom.φion ), '--.')
ax.plot(ZJ_csaa[0], ZJ_csaa[1]/interp1d(NS_tf_ne11[0], NS_tf_ne11[1])(ZJ_csaa[0])-1, ':')

ax.set_yscale('symlog', linthresh=1e-8)
ax.set_xlim(np.min(ZJ_csaa[0]),3)
# ax.set_xscale('log')
ax.grid(alpha=0.2)

## EOS

In [ ]:
def get_E(aa):
    βVeff = aa.get_βVeff(aa.φe, aa.ne, aa.ne_bar)
    η = aa.μ/aa.Te - βVeff
    βU = aa.grid.integrate_f(βVeff*aa.ne)
    
    φ_from_e_only = aa.get_φe(-aa.ne)[0] 
    φ_from_i_only = aa.get_φe(aa.ρi)[0] + aa.φion 
    U = -aa.grid.integrate_f(aa.ne * (0.5*φ_from_e_only + φ_from_i_only))

    I_onehalf = FermiDirac.Ionehalf(η)
    I_threehalf = FermiDirac.Ithreehalf(η)
    
    K = (2*aa.Te)**1.5/(2*π**2) * aa.Te * aa.grid.integrate_f( I_threehalf)
    return U, K, (U + K)

def get_P(aa):
    P = (2*aa.Te)**2.5/(6*π**2) * FermiDirac.Ithreehalf(aa.μ/aa.Te)
    return P

def print_EOS(aa):
    Ee_pot, Ee_K, Ee = get_E(aa)
    P_e =  get_P(aa)
    print(f"____________________________________________")
    print(f"Ee_pot_density = {Ee_pot*aa.ni_bar:0.3e} [au], Ee_K = {Ee_K*aa.ni_bar:0.3e} [au], Ee_tot = {Ee*aa.ni_bar:0.3e} [au]")
    print(f"P_e = {P_e:0.3e} [au], {P_e*AU_to_bar/1e6:0.3e} [Mbar]")
    print(f"Virial if {Ee_K:0.3e} = {3/2*P_e*aa.Vol - 0.5*Ee_pot:0.3e} --->  off by {100*Ee_K/(3/2*P_e*aa.Vol - 0.5*Ee_pot) - 100:0.3e} % ")
    
    print(f"\nβEe_pot/Z = {Ee_pot/aa.Te/aa.Z:0.3f}, βEe_K/Z = {Ee_K/aa.Te/aa.Z:0.3f}, βEe_tot/Z = {Ee/aa.Te/aa.Z:0.3f}")
    print(f"βP_e Ω/Z = {P_e*aa.Vol/aa.Te/aa.Z:0.3f}")
    # print(f"Virial if {Ee_K:0.3e} = {3/2*P_e*aa.Vol - 0.5*Ee_pot:0.3e}")

print_EOS(aa_ZJ)
print_EOS(aa_ZJ_cs)
print_EOS(aa_CS.core_atom)